In [ ]:
# import


In [ ]:
# calculate the average, median and  for each columm
print(f"The average for each column is:\n{df.mean()}")
print(f"The median for each column is:\n{df.median()}")
print(f"The standard deviation for each column is:\n{df.std()}")

# calculate the seasonal average, median and  for each columm
def season(dato):
    month=dato.month
    if month in [12,1,2]:
        return "Winter"
    elif month in [3,4,5]:
        return "Spring"
    elif month in [6,7,8]:
        return "Summer"
    else:
        return "Autumn"

df['season'] = df['dato'].apply(season)
print(f"The average for each season is:\n{df.groupby('season').mean()}")
print(f"The median for each season is:\n{df.groupby('season').median()}")
print(f"The standard deviation for each season is:\n{df.groupby('season').std()}")


